In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 98% 1.04G/1.06G [00:08<00:00, 236MB/s]
100% 1.06G/1.06G [00:08<00:00, 132MB/s]


In [3]:
import zipfile

with zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [4]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [5]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/test'
valid_path = '/content/test'

In [6]:
# Import the VGG16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

### Explanation:
- **`for layer in vgg16.layers:`**: This loops through each layer in the `vgg16` model.
- **`layer.trainable = False`**: This sets the `trainable` attribute of each layer to `False`, meaning that the weights in these layers will not be updated during training. Essentially, these layers become "frozen."

### Why it's used:
- This approach is typically used when you want to fine-tune a pre-trained model (like VGG16) for a new task but don't want to modify the weights of the original layers. By freezing the layers, you only train the new layers you add (e.g., the fully connected layers or the output layer).
- This helps prevent overfitting and reduces computational complexity since you don’t need to update the pre-trained layers during training.

In [8]:
  # useful for getting number of output classes
folders = glob('/content/train/*')

In [9]:
folders

['/content/train/dogs', '/content/train/cats']

### Explanation:
- **`glob('/content/train/*')`**: The `glob` function from the `glob` module returns a list of file paths matching the specified pattern. Here, the pattern `/content/train/*` means:
  - **`/content/train/`**: The directory path.
  - **`*`**: A wildcard character that matches any file or directory in the `/content/train/` folder.
  
  In this case, it retrieves a list of all files and subdirectories within the `/content/train/` directory.

### Use case:
This is commonly used in machine learning tasks where you need to load images from different categories stored in subfolders. Each subfolder represents a class or category of images. For example, in a folder structure like:
```
/content/train/
    /cat/
    /dog/
```
Running `glob('/content/train/*')` will return the paths to the `cat` and `dog` directories, which you can then process for image loading or data augmentation.

In [10]:
# our layers - you can add more if you want
x = Flatten()(vgg16.output)

The code `x = Flatten()(vgg16.output)` flattens the output of the VGG16 model (a multi-dimensional tensor) into a 1D vector to feed into fully connected layers for classification.

In [11]:
#prediction = Dense(len(folders), activation='sigmoid')(x) ## For MultiClass
# Correct output layer for binary classification
prediction = Dense(1, activation='sigmoid')(x)


The code `prediction = Dense(len(folders), activation='softmax')(x)` adds a fully connected layer (Dense layer) to the model. It has the number of units equal to the number of folders (classes), and uses the softmax activation function to output probabilities for each class.

In [12]:
# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

The code `model = Model(inputs=vgg16.input, outputs=prediction)` creates a new Keras model that uses the input from the pre-trained VGG16 model and outputs the `prediction` layer you defined earlier. This model can now be trained or used for inference, where:

- **inputs**: The input layer of the VGG16 model (`vgg16.input`).
- **outputs**: The `prediction` layer, which is the final output of the model containing class probabilities.



In [13]:
# view the structure of the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │          25,089 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 14,739,777 (56.23 MB)

 Trainable params: 25,089 (98.00 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [14]:
# tell the model what cost and optimization method to use
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [15]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [16]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 20000 images belonging to 2 classes.


In [17]:
test_set = test_datagen.flow_from_directory('/content/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 5000 images belonging to 2 classes.


In [18]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 300s 452ms/step - accuracy: 0.8634 - loss: 0.2994 - val_accuracy: 0.9344 - val_loss: 0.1647
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 48us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/20


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


625/625 ━━━━━━━━━━━━━━━━━━━━ 280s 444ms/step - accuracy: 0.9221 - loss: 0.1928 - val_accuracy: 0.9260 - val_loss: 0.1702
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 30us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 278s 439ms/step - accuracy: 0.9299 - loss: 0.1739 - val_accuracy: 0.9212 - val_loss: 0.1920
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 279s 444ms/step - accuracy: 0.9370 - loss: 0.1583 - val_accuracy: 0.9350 - val_loss: 0.1712
Epoch 8/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 35us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 323s 446ms/step - accuracy: 0.9409 - loss: 0.1494 - val_accuracy: 0.9326 - val_loss: 0.1686
Epoch 10/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 24us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 280s 445ms/step - accuracy: 0.9420 - loss: 0.1441 - val_accu